<a href="https://colab.research.google.com/github/marllinkang/deep_running/blob/main/dl8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip -qq "/content/drive/MyDrive/test _ws/deep learning/dogs-vs-cats.zip"

In [ ]:
!unzip -qq "./test1.zip"

In [ ]:
!unzip -qq "./train.zip"

In [ ]:
# 2.1 일단 우루루~ import

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#2.2 random seed 고정

In [ ]:
seed = 13
tf.random.set_seed(seed)
np.random.seed(seed)

In [ ]:
#2.3 train 파일명 정리

In [ ]:
train_df = pd.DataFrame({"file":os.listdir("/content/train")})
train_df["label"]= train_df["file"].apply(lambda x : x.split(".")[0])


train_df.head()

,file,label
0,dog.6931.jpg,dog
1,dog.11978.jpg,dog
2,dog.9839.jpg,dog
3,cat.1000.jpg,cat
4,dog.3563.jpg,dog


In [ ]:
#2.5 train test split

In [ ]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(train_df, test_size=0.2,stratify = train_df["label"], random_state= 13)

In [ ]:
#2.6 잠깐 data augmentation 데이터 증강

In [ ]:
#2.7 ImageDataGenerator 사용

In [ ]:
# from keras.api._v2.keras import preprocessing
train_datagen = ImageDataGenerator(
    rotation_range = 15,
    horizontal_flip = True,
    preprocessing_function = preprocess_input
)

val_datagen = ImageDataGenerator(preprocessing_function= preprocess_input)

In [ ]:
#2.8 이번에는 data frame에서 train 데이터

In [ ]:
FILES = "/content/"
batch_size = 160
train_generator = train_datagen.flow_from_dataframe(
    dataframe = train_data,
    directory = FILES +"train/",
    x_col = "file",
    y_col = "label",
    class_mode = "categorical",
    target_size = (224, 224),
    batch_size = batch_size,
    seed = 13
)

Found 20000 validated image filenames belonging to 2 classes.


In [ ]:
val_generator = val_datagen.flow_from_dataframe(
    dataframe = val_data,
    directory = FILES +"train/",
    x_col = "file",
    y_col = "label",
    class_mode = "categorical",
    target_size = (224, 224),
    batch_size = batch_size,
    seed = 13,
    shuffle = False

)

Found 5000 validated image filenames belonging to 2 classes.


In [ ]:
#2.10 VGG를 부르는데 top을 포함하지 않는다?

In [ ]:
base_model = VGG16(
    weights = "imagenet",
    input_shape = (224, 224, 3),
    include_top = False
)

In [ ]:
#2.11 summary로 모델을 보면

In [ ]:
base_model.summary()
#2.13 원래는 마지막에 이게 있어야 하거덩

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
#2.14 그리고 가중치가 학습되지 않도록 잠궈버리자

In [ ]:
for layers in base_model.layers:
    layers.trainable = False

base_model.summary()
#2.15 이 단계까지는 summary로 보면 학습가능한 가중치가 없다

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
#2.16 여기를 잘라서(끝부분)
#2.17 나의 모델을 뒤에 붙이자

In [ ]:
# from keras.src.metrics.confusion_metrics import activations
def vgg16_pretrained():

    model = Sequential(
        [
            base_model,
            GlobalAveragePooling2D(),
            Dense(100, activation = "relu"),
            Dropout(0.4),
            Dense(64, activation = 'relu'),
            Dense(2, activation = "softmax")

        ]
    )

    return model

In [ ]:
#2.18 살짝 메모리 비움

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
#2.19 모델 재구성

In [ ]:
model = vgg16_pretrained()
model.compile(loss = "categorical_crossentropy",optimizer = "adam",metrics = "accuracy")
model.summary()
#2.20 학습할 변수가 적다. - 그렇다고 시간이 만만하진 않음

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (  (None, 512)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 100)               51300     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                6464      
                                                                 
 dense_2 (Dense)             (None, 2)                 130       
                                                        

In [ ]:
#2.21 콜백 ReduceLROnPlateau

# 모델의 개선이 없을 경우, Learning Rate를 조절해 모델의 개선을 유도
# monitor : 만약 'val_loss'를 입력하면 val_loss가 더이상 감소되지 않을 경우 ReduceLROnPlateau을 적용
# factor : Learning rate를 얼마나 감소시킬 지

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor = "val_accuracy",
    patience = 2,
    verbose = 1,
    factor = 0.5,
    min_lr = 0.000000001
)

In [ ]:
#2.22 콜백 EarlyStopping

# 모델을 더 이상 학습을 못할 경우(loss, metric등의 개선이 없을 경우), 학습 도중 미리 학습을 종료
# patience : Training이 진행됨에도 더이상 monitor되는 값의 개선이 없을 경우, 최적의 monitor 값을 기준으
# 로 몇 번의 epoch을 진행할 지 정하는 값

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = "val_accuracy",
    patience = 5,
    verbose = 1,
    mode = "max"
)

In [ ]:
#2.23 CheckPoint
#모델을 저장할 때 사용

In [ ]:

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    monitor = "val_accuracy",
    filepath = "catdog_vgg16_.{epoch:02d}-[val_accuracy:.6f].hdf5",
    verbose = 1,
    save_best_only = True,
    save_weights_only = True
)

In [ ]:
# 2.24 학습 진행

In [ ]:
history = model.fit(
    train_generator,
    epochs = 3,
    validation_data = val_generator,
    validation_steps = val_data.shape[0] // batch_size,
    steps_per_epoch = train_data.shape[0] // batch_size,
    callbacks = [reduce_lr, early_stopping, checkpoint]
)

Epoch 1/3
 35/125 [=======>......................] - ETA: 2:31:47 - loss: 0.4537 - accuracy: 0.8763

In [ ]:
#2.25 조금 길지만~

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig,  axes = plt.subplots(1, 2, figsize = (12,4))

sns.lineplot(x= range(len(history.history["loss"])),
             y = history.history["loss"], ax= axes[0],
             label = "Training Loss")
sns.lineplot(x= range(len(history.history["loss"])),
             y = history.history["val_loss"], ax= axes[0],
             label = "Validation Loss")

sns.lineplot(x= range(len(history.history["accuracy"])),
             y = history.history["accuracy"], ax= axes[1],
             label = "Training Accuracy")
sns.lineplot(x= range(len(history.history["accuracy"])),
             y = history.history["val_accuracy"], ax= axes[1],
             label = "validation Accuracy")
axes[0].set_title("Loss"); axes[1].set_title("Accuracy")

sns.despine()
plt.show()



In [ ]:
#2.27 테스트 데이터

In [ ]:
val_pred = model.predict(val_generator,steps = np.ceil(val_data.shape[0] /batch_size))

val_data.lov[: "val_pred"] = np.argmax(val_pred, axis = 1)

labels = dict((v,k) for k, v in val_generarot.class_indices.items())

val_data.loc[:, "val_pred"] = val_data.loc[:,"val_pred"].map(labels)

In [ ]:
#2.28 Confusion을 열어보고

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

fig, ax = plt.subplots(figsize = (9,6))

cm = confusion_matrix(val_data["label"], val_data["val_pred"])

disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = ["cat", "dog"])
disp.plot(cmap = plt.cmBlues, ax = ax)

ax.set_title("validation Set")
plt.show()

In [ ]:
#2.30 틀린아이들만

In [ ]:
val_errors = val_data[(val_data.label) != (val_data.val_pred)].reset_index(drop = True)
val_errors

In [ ]:
#2.31 그림을 그려보자

In [ ]:
fig = plt.figure(1, figsize = (24, 20))

for i in range(81):
    if i == len(val_errors):
        break
    plt.subplot(9, 9, i+1)
    image = plt.imeread(".")